In [ ]:
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

import symforce.symbolic as sf

epsilon = 1e-9

import numpy as np
import plotly.graph_objects as go

frame_count = 100
loops = 7
slider_steps = 50

### Helper Functions for Generating Figures

In [ ]:
def get_tangent_vectors(a: sf.Unit3):
    basis = a.basis(epsilon)
    return basis[:, 0].to_numpy().flatten(), basis[:, 1].to_numpy().flatten()

In [ ]:
# Generate an animation that rotates around the unit sphere.


def create_frames(num_frames):
    frames = []
    for i in range(num_frames):
        v = get_unit_vector(i)
        v_draw = v.to_unit_vector().to_numpy().flatten()
        t1, t2 = get_tangent_vectors(v)
        frame_data = [
            create_unit_sphere(),
            create_vector_trace(np.zeros(3), v_draw, "Unit3"),
            create_vector_trace(v_draw, t1, "Tangent X"),
            create_vector_trace(v_draw, t2, "Tangent Y"),
        ]
        frames.append(go.Frame(data=frame_data, name=str(i)))
    return frames


def get_unit_vector(i):
    phi = np.pi * i / frame_count
    theta = 2 * loops * np.pi * i / frame_count
    return sf.Unit3(sf.V3(np.cos(phi), np.sin(phi) * np.cos(theta), np.sin(phi) * np.sin(theta)))


def create_vector_trace(a: np.array, b: np.array, name: str = None, color: str = None):
    return go.Scatter3d(
        x=[a[0], a[0] + b[0]],
        y=[a[1], a[1] + b[1]],
        z=[a[2], a[2] + b[2]],
        mode="lines+markers",
        marker=dict(size=3),
        line=dict(width=3, color=color),
        name=name,
    )


def create_unit_sphere():
    u = np.linspace(0, 2 * np.pi, 20)
    v = np.linspace(0, np.pi, 20)

    x = np.outer(np.cos(u), np.sin(v))

    # Plot the sphere with a slight grey gradient. and transparent.
    return go.Surface(
        x=x,
        y=np.outer(np.sin(u), np.sin(v)),
        z=np.outer(np.ones(np.size(u)), np.cos(v)),
        colorscale=[[0, "rgb(220,220,220)"], [1, "rgb(220,220,220)"]],
        opacity=0.5,
        showscale=False,
    )


def build_animation():
    initial_v = get_unit_vector(0)
    t1, t2 = get_tangent_vectors(initial_v)

    v_draw = initial_v.to_unit_vector().to_numpy().flatten()

    frames = create_frames(frame_count)

    # Define slider steps.
    slider_steps = []
    for i, frame in enumerate(frames):
        step = {
            "method": "animate",
            "label": f"{i}",
            "args": [
                [frame.name],
                {
                    "mode": "immediate",
                    "frame": {"redraw": True, "duration": 0},
                    "transition": {"duration": 0},
                },
            ],
        }
        slider_steps.append(step)

    fig = go.Figure(
        data=[
            create_unit_sphere(),
            create_vector_trace(np.zeros(3), v_draw, "Unit3", "black"),
            create_vector_trace(v_draw, t1, "Tangent X", "blue"),
            create_vector_trace(v_draw, t2, "Tangent Y", "red"),
        ],
        layout=go.Layout(
            scene=dict(
                xaxis=dict(range=[-1.5, 1.5]),
                yaxis=dict(range=[-1.5, 1.5]),
                zaxis=dict(range=[-1.5, 1.5]),
                aspectmode="cube",
            ),
            sliders=[
                {
                    "active": 0,
                    "currentvalue": {"prefix": "Frame: "},
                    "pad": {"t": 50},
                    "steps": slider_steps,
                }
            ],
        ),
        frames=frames,
    )

    fig.show()

In [ ]:
# Create a figure that just shows the the tangent x / y vectors across the unit sphere.


def compute_unit_vectors(n):
    """
    Returns n unit vectors evenly distributed on a sphere.
    """
    points = []
    golden_angle = np.pi * (3 - np.sqrt(5))
    for i in range(n):
        y = 1 - (i / float(n - 1)) * 2  # y goes from 1 to -1
        radius = np.sqrt(1 - y * y)
        theta = golden_angle * i
        x = np.cos(theta) * radius
        z = np.sin(theta) * radius
        points.append(sf.Unit3(sf.V3(x, y, z)))
    return points


def group_vector_traces(vectors, color, name):
    """
    Given vectors as list of (start, end) pairs (each a 3-array),
    returns a single trace that draws them as separate segments.
    """
    xs, ys, zs = [], [], []
    # For each segment, append start and end coords, then a gap (None).
    for start, end in vectors:
        # Start point.
        xs.append(start[0])
        ys.append(start[1])
        zs.append(start[2])
        # End point.
        xs.append(end[0])
        ys.append(end[1])
        zs.append(end[2])
        # Insert gap.
        xs.append(None)
        ys.append(None)
        zs.append(None)
    return go.Scatter3d(
        x=xs,
        y=ys,
        z=zs,
        mode="lines+markers",
        marker=dict(size=3),
        line=dict(width=3, color=color),
        name=name,
    )


def build_tangent_figure():
    unit_vectors = compute_unit_vectors(frame_count)

    data = [create_unit_sphere()]

    vectors_x = []
    vectors_y = []
    for v in unit_vectors:
        v_draw = v.to_unit_vector().to_numpy().flatten()
        t1, t2 = get_tangent_vectors(v)
        vectors_x.append((v_draw, v_draw + t1 * 0.1))
        vectors_y.append((v_draw, v_draw + t2 * 0.1))

    data.append(group_vector_traces(vectors_x, "blue", "Tangent X"))
    data.append(group_vector_traces(vectors_y, "red", "Tangent Y"))

    # Adjust the size to help visualize the tangent vectors.
    fig = go.Figure(
        data=tuple(data),
        layout=go.Layout(
            scene=dict(
                xaxis=dict(range=[-1.5, 1.5]),
                yaxis=dict(range=[-1.5, 1.5]),
                zaxis=dict(range=[-1.5, 1.5]),
                aspectmode="cube",
            ),
        ),
    )

    fig.show()


def build_tangent_figure_singularity():
    # Only select unit vectors around the singularity direction at (1, 0, 0).
    unit_vectors = []
    for i in range(frame_count):
        golden_angle = np.pi * (3 - np.sqrt(5))
        phi = 0.05 * np.pi * i / frame_count
        theta = 2 * i * 100 * golden_angle / frame_count
        unit_vectors.append(
            sf.Unit3(sf.V3(np.cos(phi), np.sin(phi) * np.cos(theta), np.sin(phi) * np.sin(theta)))
        )
    # unit_vectors = compute_unit_vectors(frame_count)
    data = [create_unit_sphere()]
    vectors_x = []
    vectors_y = []
    for v in unit_vectors:
        v_draw = v.to_unit_vector().to_numpy().flatten()
        t1, t2 = get_tangent_vectors(v)
        vectors_x.append((v_draw, v_draw + t1 * 0.01))
        vectors_y.append((v_draw, v_draw + t2 * 0.01))

    data.append(group_vector_traces(vectors_x, "blue", "Tangent X"))
    data.append(group_vector_traces(vectors_y, "red", "Tangent Y"))

    # Adjust the size to help visualize the tangent vectors.
    fig = go.Figure(
        data=tuple(data),
        layout=go.Layout(
            scene=dict(
                xaxis=dict(range=[-1.5, 1.5]),
                yaxis=dict(range=[-1.5, 1.5]),
                zaxis=dict(range=[-1.5, 1.5]),
                aspectmode="cube",
            ),
        ),
    )
    fig.show()

### Display Unit3 Tangent Space Sweep

In [ ]:
build_animation()

### Display Unit3 Tangent X / Y Space

In [ ]:
build_tangent_figure()

### Display Unit3 Tangent X / Y Space -- Around Singularity

In [ ]:
build_tangent_figure_singularity()